In [1]:
import sys
from bs4 import BeautifulSoup
import scrapy
from lxml.etree import fromstring
import requests
import time
import re
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from six.moves.urllib import parse
import selenium.webdriver.chrome.service as service
from pyvirtualdisplay import Display
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def scroll_page(url):
    driver = webdriver.Chrome('/Users/mohan/Documents/medius_health/chromedriver')
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    html = driver.find_element_by_class_name('css-y1gt6f').get_attribute('innerHTML')
    return html
def get_soup(x):
    soup = BeautifulSoup(x,'lxml')
    return soup

In [3]:
def extract_name(x):
    name = x.find('h2',{'class':'h1 css-kjyn3a egb32mv1'})
    name = name.text
    return name
def extract_url(x):
    url = x.find('a',attrs = {'href':re.compile('/health/*')})
    url = url.get('href')
    complete_url = 'https://www.healthline.com'+url
    return complete_url

In [4]:
def extract_details(x):
    name = extract_name(x)
    complete_url = extract_url(x)
    response = requests.get(complete_url)
    source_code = response.text
    sub_soup = get_soup(source_code)
    symptoms = sub_soup.find('a',attrs = {'name':re.compile('symptoms*')})
    #symptoms_list = []
#     if symptoms:
#         counter = symptoms.findNext('ul').contents
#         for i in range(len(counter)):
#             if counter[i] != ' ':
#                 symptoms_list.append(symptoms.findNext('ul').contents[i].string)
#         symptoms_list = list(filter(None.__ne__, symptoms_list))

    return name,complete_url,symptoms

In [5]:
start_url = 'https://www.healthline.com/symptom/dizziness'

In [6]:
def extract_symptoms(symptoms):
    symptoms_list = []
    counter = symptoms.findNext('ul').contents
    for i in range(len(counter)):
        if counter[i] != ' ':
            symptoms_list.append(symptoms.findNext('ul').contents[i].string)
    symptoms_list = list(filter(None.__ne__, symptoms_list))
    return symptoms_list

In [7]:
final_symptoms = {}
def start_scraping(soup):
    for disease in soup.findAll('li', {'class': 'css-ee6cf6'}):
        name,complete_url,symptoms = extract_details(disease)
        print(name)
        print(complete_url)
        if symptoms:
            symptoms_list = []
            symptoms_list = extract_symptoms(symptoms)
            final_symptoms[name] = symptoms_list
            #pass
            print(symptoms_list)
        else:
            #sub_html = scroll_page(complete_url)
            response = requests.get(complete_url)
            source_code = response.text
            sub_soup = get_soup(source_code)
            start_scraping(sub_soup)


In [8]:
homepage_html = scroll_page(start_url)
soup = get_soup(homepage_html)
start_scraping(soup)

Low Blood Sugar (Hypoglycemia)
https://www.healthline.com/health/hypoglycemia
['blurry vision', 'rapid heartbeat', 'sudden mood changes', 'sudden nervousness', 'pale skin', 'headache', 'hunger', 'shaking', 'dizziness', 'sweating', 'difficulty sleeping', 'skin tingling']
Labyrinthitis
https://www.healthline.com/health/labyrinthitis
['dizziness', 'vertigo', 'loss of balance', 'nausea and vomiting', 'loss of hearing in the high-frequency range in one ear', 'difficulty focusing your eyes']
What Is Hypotension?
https://www.healthline.com/health/hypotension
Orthostatic Hypotension (Postural Hyoptension)
https://www.healthline.com/health/orthostatic-hypotension
Dehydration
https://www.healthline.com/health/dehydration
Fever
https://www.healthline.com/health/fever
Common Cold Overview
https://www.healthline.com/health/common-cold
['adults over age 65', 'women who are pregnant', 'children under age 5', 'people with weakened immune systems due to chronic illness']
Cold and Flu
https://www.health

KeyboardInterrupt: 

In [9]:
final_symptoms

{'Alcohol-Related Liver Disease': ['nausea',
  'loss of appetite',
  'jaundice',
  'fatigue',
  'abdominal discomfort',
  'increased thirst',
  'swelling in the legs and abdomen',
  'weight loss',
  'darkening or lightening of the skin',
  'red hands or feet',
  'dark bowel movements',
  'fainting',
  'unusual agitation',
  'mood swings',
  'confusion',
  'bleeding gums',
  'enlarged breasts (in men)'],
 'All About Pneumonia and How to Treat It Effectively': ['fever, sweating, and chills',
  'shortness of breath',
  'chest pain'],
 'Bacterial Gastroenteritis': ['loss of appetite',
  'nausea and vomiting',
  'diarrhea',
  'abdominal pains and cramps',
  'blood in your stools',
  'fever'],
 'Bone Infection (Osteomyelitis)': ['drainage from the area'],
 'Caffeine Overdose: How Much Is Too Much?': ['dizziness',
  'diarrhea',
  'increased thirst',
  'insomnia',
  'headache',
  'fever',
  'irritability'],
 'Cold and Flu': ['adults over age 65',
  'women who are pregnant',
  'children under a

In [26]:
#json = json.dumps(final_symptoms)

In [15]:
with open('test.json','w') as output_file:
    output_file.write(json)

In [31]:
for key,values in final_symptoms.items():
    if 'dizziness' in values:
        print(key)

Low Blood Sugar (Hypoglycemia)
Labyrinthitis
Caffeine Overdose: How Much Is Too Much?
Heat Emergencies


In [32]:
for key,values in final_symptoms.items():
    if 'dizziness' and 'rapid heartbeat' in values:
        print(key)

Low Blood Sugar (Hypoglycemia)
